In [2]:
import pandas as pd
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from hazm import Normalizer




In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz


--2025-06-11 12:49:46--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1258183862 (1.2G) [binary/octet-stream]
Saving to: ‘cc.fa.300.vec.gz’

cc.fa.300.vec.gz    100%[===================>]   1.17G   263MB/s    in 8.7s    

2025-06-11 12:49:55 (138 MB/s) - ‘cc.fa.300.vec.gz’ saved [1258183862/1258183862]



In [3]:
!gunzip cc.fa.300.vec.gz


In [4]:
!pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
bigframes 2.5.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is 

In [ ]:
!pip install --upgrade gensim numpy scipy

In [1]:
!pip install gensim scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
t

#### Fast Text Embedding

In [2]:
from gensim.models import KeyedVectors

ft_model = KeyedVectors.load_word2vec_format('cc.fa.300.vec', binary=False)


In [3]:
def get_sentence_vector_fasttext(sentence, model):
    words = sentence.split()
    vectors = [model[word] for word in words if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

In [6]:
df = pd.read_csv('/content/Preprocessed_data.csv')
df.head()

,tweet_with_emoji_meaning,label
0,پیرمرد وصیت احدی تاکید احدی درد دل نکنید بعدش ...,0
1,مجوز بده ملت ماشین استاندارد بتونن بیارن سوار...,0
2,دیت دختره دید زشتم می‌خواست پاشه بره آینهی دست...,1
3,اکیپ دخترونه هست پایه قراراست کلاس نمیذاره زود...,0
4,۵۰۰ نفری مراسم سالگرد پدر همسرم شرکت فوتی‌ی ۴۰...,0


In [7]:
texts = df["tweet_with_emoji_meaning"].astype(str).tolist()
labels = df["label"].tolist()

In [8]:
X = np.array([get_sentence_vector_fasttext(text, ft_model) for text in texts])
y = np.array(labels)

In [9]:
X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
models = {
    'Logistic Regression': LogisticRegression(),
    # 'MultinomialNB': MultinomialNB(),
    'SVC': SVC(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
}


In [15]:
def cacl_metrics(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print('Accuracy: ',   accuracy_score(y_test, y_pred))


In [16]:
for model_name, model in models.items():
    print(f"Model: {model_name}")
    # if model_name == 'MultinomialNB':
    cacl_metrics(model, X_train_ft, X_test_ft, y_train_ft, y_test_ft)
    # else:

        # cacl_metrics(model, x_train_svd, x_test_svd, y_train, y_test)
    print("-"*100)


Model: Logistic Regression
              precision    recall  f1-score   support

           0       0.76      0.79      0.78      1543
           1       0.76      0.73      0.74      1393

    accuracy                           0.76      2936
   macro avg       0.76      0.76      0.76      2936
weighted avg       0.76      0.76      0.76      2936

Accuracy:  0.7602179836512262
----------------------------------------------------------------------------------------------------
Model: SVC
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1543
           1       0.76      0.79      0.77      1393

    accuracy                           0.78      2936
   macro avg       0.78      0.78      0.78      2936
weighted avg       0.78      0.78      0.78      2936

Accuracy:  0.782016348773842
----------------------------------------------------------------------------------------------------
Model: Random Forest
              precision   